<a href="https://colab.research.google.com/github/ntolayd/CE888/blob/main/Lab8/Lab8_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip "/content/gdrive/My Drive/data.zip"


In [3]:
import numpy as np
from tqdm import tqdm
from random import shuffle
import cv2
import os 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Flatten, Conv2D, MaxPooling2D
from keras.layers import Dropout
from keras.layers import Dense
from keras.optimizers import Adam, RMSprop
from sklearn.metrics import classification_report

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=45,
                                   #horizontal_flip = True,
                                   vertical_flip=True,
                                   #width_shift_range=0.2,
                                   #height_shift_range=0.2,
                                   #fill_mode='reflect',
                                   #brightness_range=[0.5, 1.5]
                                   #preprocessing_function=noise, 
                                   validation_split=0.2
                                   )

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'data',
    target_size=(150, 150),
    batch_size=64,
    color_mode='rgb',
    class_mode='categorical',
    ) # set as training data

validation_generator = train_datagen.flow_from_directory(
    'data', 
    target_size=(150, 150),
    color_mode='rgb',
    batch_size=64,
    class_mode='categorical', subset='validation') # set as validation data


Found 808 images belonging to 4 classes.
Found 160 images belonging to 4 classes.


traning with freezing layers

In [13]:
from keras.applications import DenseNet121
densenet_v2 = DenseNet121(
    include_top=False,
    input_shape=(150,150,3))
model = Sequential()
model.add(densenet_v2)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

for layer in model.layers[:-3]:
	layer.trainable = False


model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.000075),
              metrics=['accuracy'])

In [14]:
history = model.fit(
          train_generator,
          steps_per_epoch = train_generator.samples // 64,
          validation_data = validation_generator, 
          validation_steps = validation_generator.samples // 64,
          epochs = 15)

Epoch 1/15
12/12 [==============================] - 14s 685ms/step - loss: 3.0353 - accuracy: 0.3234 - val_loss: 2.1075 - val_accuracy: 0.3750
Epoch 2/15
12/12 [==============================] - 6s 521ms/step - loss: 2.9470 - accuracy: 0.2575 - val_loss: 2.0610 - val_accuracy: 0.3828
Epoch 3/15
12/12 [==============================] - 6s 509ms/step - loss: 2.6780 - accuracy: 0.2957 - val_loss: 1.8524 - val_accuracy: 0.3281
Epoch 4/15
12/12 [==============================] - 6s 521ms/step - loss: 2.5243 - accuracy: 0.3279 - val_loss: 1.7523 - val_accuracy: 0.4297
Epoch 5/15
12/12 [==============================] - 6s 511ms/step - loss: 2.7183 - accuracy: 0.3062 - val_loss: 1.5836 - val_accuracy: 0.3672
Epoch 6/15
12/12 [==============================] - 6s 512ms/step - loss: 2.5622 - accuracy: 0.3146 - val_loss: 1.5184 - val_accuracy: 0.4062
Epoch 7/15
12/12 [==============================] - 6s 513ms/step - loss: 2.4167 - accuracy: 0.3691 - val_loss: 1.5408 - val_accuracy: 0.5000
Epoch

Training whole model

In [16]:
from keras.applications import DenseNet121
densenet_v2 = DenseNet121(
    include_top=False,
    input_shape=(150,150,3))
model = Sequential()
model.add(densenet_v2)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.000075),
              metrics=['accuracy'])

In [17]:
history = model.fit(
          train_generator,
          steps_per_epoch = train_generator.samples // 64,
          validation_data = validation_generator, 
          validation_steps = validation_generator.samples // 64,
          epochs = 15)

Epoch 1/15
12/12 [==============================] - 26s 733ms/step - loss: 1.7220 - accuracy: 0.4205 - val_loss: 0.8194 - val_accuracy: 0.7188
Epoch 2/15
12/12 [==============================] - 7s 560ms/step - loss: 0.5923 - accuracy: 0.7799 - val_loss: 0.6476 - val_accuracy: 0.7344
Epoch 3/15
12/12 [==============================] - 7s 559ms/step - loss: 0.3820 - accuracy: 0.8647 - val_loss: 0.5618 - val_accuracy: 0.8047
Epoch 4/15
12/12 [==============================] - 7s 563ms/step - loss: 0.2666 - accuracy: 0.9001 - val_loss: 0.2554 - val_accuracy: 0.9062
Epoch 5/15
12/12 [==============================] - 7s 559ms/step - loss: 0.2135 - accuracy: 0.9239 - val_loss: 0.3159 - val_accuracy: 0.9062
Epoch 6/15
12/12 [==============================] - 7s 561ms/step - loss: 0.2045 - accuracy: 0.9352 - val_loss: 0.3226 - val_accuracy: 0.8906
Epoch 7/15
12/12 [==============================] - 7s 565ms/step - loss: 0.1485 - accuracy: 0.9478 - val_loss: 0.2129 - val_accuracy: 0.9219
Epoch

As it can be seem from the results of 2 models, there is a significant difference between model scores on validation data. When we freeze some layers in the densenet model, we got around 57 percent of accuracy. However, when we train the whole model without freezing, we achieve 99 percent of accuracy on the validation data. 